In [1]:
# A warning message from DGL will appear which seems to be related to an open issue in the DGL library, 
# it won't hurt the execution of the program, please ignore it.
import quartz

Using backend: pytorch[21:06:36] /opt/dgl/src/runtime/tensordispatch.cc:43
: TensorDispatcher: dlopen failed: /home/zikunli/anaconda3/envs/quantum/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory


## Construct QuartzContext
The following lines are used to construct a context in quartz which contains all the generated transfers under a given gate set.

In [2]:
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 't', 'tdg'], filename='../bfs_verified_simplified.json')

In [3]:
quartz_context.num_xfers

3587

In [4]:
quartz_context.get_xfers()

 ...]

In [5]:
quartz_context.get_xfer_from_id(id=0)

## QASM parser
You can construct a PyQASMParser to generate a PyDAG from a QASM file.

In [4]:
parser = quartz.PyQASMParser(context=quartz_context)

In [5]:
my_dag = parser.load_qasm(filename="barenco_tof_3_opt_path/subst_history_39.qasm")
my_dag.num_qubits, my_dag.num_gates

(5, 58)

## PyGraph

You can construct a PyGraph from a PyDAG.

Below are some examples showing the APIs in PyGraph.

In [6]:
my_graph = quartz.PyGraph(context=quartz_context, dag=my_dag)

In [9]:
my_graph.num_nodes, my_graph.num_edges

(58, 77)

In [10]:
my_graph.all_edges()

[(0, 1, 0, 1),
 (1, 2, 1, 0),
 (1, 5, 0, 0),
 (2, 3, 0, 1),
 (3, 4, 1, 0),
 (3, 7, 0, 0),
 (4, 5, 0, 1),
 (5, 6, 1, 0),
 (5, 8, 0, 1),
 (6, 7, 0, 1),
 (7, 8, 0, 0),
 (7, 9, 1, 0),
 (8, 10, 1, 0),
 (8, 11, 0, 0),
 (9, 27, 0, 1),
 (10, 11, 0, 1),
 (11, 12, 0, 0),
 (11, 13, 1, 0),
 (12, 31, 0, 0),
 (13, 14, 0, 0),
 (14, 15, 0, 1),
 (15, 16, 1, 0),
 (15, 19, 0, 0),
 (16, 17, 0, 1),
 (17, 18, 1, 0),
 (17, 21, 0, 0),
 (18, 19, 0, 1),
 (19, 20, 1, 0),
 (19, 22, 0, 1),
 (20, 21, 0, 1),
 (21, 22, 0, 0),
 (21, 23, 1, 0),
 (22, 24, 1, 0),
 (22, 26, 0, 0),
 (23, 25, 0, 0),
 (24, 26, 0, 1),
 (25, 27, 0, 0),
 (26, 28, 0, 0),
 (26, 29, 1, 0),
 (27, 30, 1, 0),
 (27, 33, 0, 0),
 (28, 46, 0, 0),
 (29, 44, 0, 0),
 (30, 31, 0, 1),
 (31, 32, 1, 0),
 (31, 35, 0, 0),
 (32, 33, 0, 1),
 (33, 34, 1, 0),
 (33, 36, 0, 1),
 (34, 35, 0, 1),
 (35, 36, 0, 0),
 (35, 37, 1, 0),
 (36, 38, 1, 0),
 (36, 40, 0, 0),
 (37, 39, 0, 0),
 (38, 40, 0, 1),
 (40, 41, 0, 0),
 (40, 42, 1, 0),
 (42, 43, 0, 0),
 (43, 44, 0, 1),
 (44, 4

In [11]:
my_graph.all_nodes()

In [12]:
my_graph.all_nodes_with_id()

[{'id': 0, 'node': <quartz.core.PyNode at 0x7f06f77d26d0>},
 {'id': 1, 'node': <quartz.core.PyNode at 0x7f06f77d2710>},
 {'id': 2, 'node': <quartz.core.PyNode at 0x7f06f77d26f0>},
 {'id': 3, 'node': <quartz.core.PyNode at 0x7f06f77d2770>},
 {'id': 4, 'node': <quartz.core.PyNode at 0x7f06f77d2790>},
 {'id': 5, 'node': <quartz.core.PyNode at 0x7f06f77d26b0>},
 {'id': 6, 'node': <quartz.core.PyNode at 0x7f06f77d27b0>},
 {'id': 7, 'node': <quartz.core.PyNode at 0x7f06f77d27d0>},
 {'id': 8, 'node': <quartz.core.PyNode at 0x7f06f77d27f0>},
 {'id': 9, 'node': <quartz.core.PyNode at 0x7f06f77d2810>},
 {'id': 10, 'node': <quartz.core.PyNode at 0x7f06f77d2830>},
 {'id': 11, 'node': <quartz.core.PyNode at 0x7f06f77d2850>},
 {'id': 12, 'node': <quartz.core.PyNode at 0x7f06f77d2870>},
 {'id': 13, 'node': <quartz.core.PyNode at 0x7f06f77d2890>},
 {'id': 14, 'node': <quartz.core.PyNode at 0x7f06f77d28b0>},
 {'id': 15, 'node': <quartz.core.PyNode at 0x7f06f77d28d0>},
 {'id': 16, 'node': <quartz.core.P

In [13]:
my_graph.get_node_from_id(id=0)

In [14]:
my_graph.to_qasm(filename="test.qasm")

In [15]:
my_graph_dgl = my_graph.to_dgl_graph()

In [16]:
my_graph_dgl.num_edges()

154

In [17]:
my_graph_dgl.edata

{'src_idx': tensor([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0]), 'dst_idx': tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0

In [18]:
my_graph_dgl.ndata

{'gate_type': tensor([ 0,  6, 14,  6, 13,  6, 14,  6,  6, 13, 14,  6, 13, 13,  0,  6, 14,  6,
        13,  6, 14,  6,  6, 13, 14,  0,  6,  6, 13, 13, 13,  6, 14,  6, 13,  6,
         6, 14, 13,  0,  6, 14, 14,  0,  6, 13,  6, 14,  6, 13,  6,  6, 14, 13,
         0,  6, 14, 14])}

In [19]:
my_graph_dgl.num_edges()

154

In [20]:
all_nodes = my_graph.all_nodes()
all_nodes

The codes below shows the usage of the two APIs `PyGraph.available_xfers` and `PyGraph.apply_xfer`.

In [21]:
# available_xfer_matrix = my_graph.get_available_xfers_matrix(context=quartz_context)
# for node in all_nodes:
#     print(my_graph.available_xfers(context=quartz_context, node=node))

In [22]:
new_graph = my_graph.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=3259), node=all_nodes[0])

In [23]:
new_graph.hash()

3885736051714220776

In [24]:
new_graph.gate_count

60

The codes below is a back tracking search implemented with the quartz python APIs.

In [7]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

def check(graph):
    graph.to_qasm(filename='best.qasm')
    qc_origin = QuantumCircuit.from_qasm_file('barenco_tof_3_opt_path/subst_history_39.qasm')
    qc_optimized = QuantumCircuit.from_qasm_file('best.qasm')
    return Statevector.from_instruction(qc_origin).equiv(Statevector.from_instruction(qc_optimized))

In [27]:
# Optimizing with BFS
import heapq
from concurrent.futures import ProcessPoolExecutor
import copy

candidate_hq = []
# heapq.heappush(candidate_hq, my_graph)
heapq.heappush(candidate_hq, (my_graph, []))
hash_set = set()
hash_set.add(my_graph.hash())
best_graph = my_graph
best_graph_trace = []
best_gate_cnt = my_graph.gate_count
q_max_len = 1000

budget = 1_000_000

# while candidate_hq != [] and budget >= 0:
#     first_candidate = heapq.heappop(candidate_hq)
#     print(first_candidate.gate_count)
#     try:
#         all_nodes = first_candidate.all_nodes()
#     except:
#         print(first_candidate)
#         print(first_candidate.gate_count)
#         print(first_candidate.num_nodes)
#         exit(1)
#     for node in all_nodes:
#         appliable_xfers = first_candidate.available_xfers(context=quartz_context, node=node)
#         for xfer in appliable_xfers:
#             new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
#             if new_graph.hash() not in hash_set:
#                 hash_set.add(new_graph.hash())
#                 heapq.heappush(candidate_hq, new_graph)
#                 if new_graph < best_graph:
#                     best_graph = new_graph
#                     best_gate_cnt = new_graph.gate_count
#                 budget -= 1
#                 if budget % 1_000 == 0:
#                     print(f'{budget}: minimum gate count is {best_gate_cnt}')

while candidate_hq != [] and budget >= 0:
    first = heapq.heappop(candidate_hq)
    first_candidate = first[0]
    first_candidate_trace = first[1]
    all_nodes = first_candidate.all_nodes()
    
    def ax(i):
        node = all_nodes[i]
        return first_candidate.available_xfers(context=quartz_context, node=node)
    
    with ProcessPoolExecutor(max_workers=24) as executor:
        results = executor.map(ax, list(range(len(all_nodes))))
        appliable_xfers_nodes = []
        for r in results:
            appliable_xfers_nodes.append(r)
        
    for i in range(len(all_nodes)):
        node = all_nodes[i]
        appliable_xfers = appliable_xfers_nodes[i]
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            if new_graph.hash() not in hash_set:
                hash_set.add(new_graph.hash())
                new_graph_trace = copy.deepcopy(first_candidate_trace)
                new_graph_trace.append((i, xfer))
                heapq.heappush(candidate_hq, (new_graph, new_graph_trace))
                if len(candidate_hq) > q_max_len:
                    candidate_hq = candidate_hq[:-1]
                if new_graph < best_graph:
                    best_graph = new_graph
                    best_graph_trace = copy.deepcopy(new_graph_trace)
                    best_gate_cnt = new_graph.gate_count
                elif new_graph.gate_count == best_graph.gate_count:
                    if len(new_graph_trace) < len(best_graph_trace):
                        best_graph = new_graph
                        best_graph_trace = copy.deepcopy(new_graph_trace)
                if best_gate_cnt == 40:
                    break
                budget -= 1
                if budget % 10_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}, correctness: {check(best_graph)}')
                    print(f'Best graph trace: {best_graph_trace}')


990000: minimum gate count is 58, correctness: True
Best graph trace: []
980000: minimum gate count is 58, correctness: True
Best graph trace: []
970000: minimum gate count is 54, correctness: True
Best graph trace: [(28, 3223), (22, 3287), (7, 440), (11, 3224), (9, 3224), (3, 1777), (21, 1173), (0, 3223), (19, 440), (24, 3223), (3, 3224), (33, 440), (19, 440), (0, 3223), (37, 3155), (19, 440), (35, 3155), (3, 3224), (19, 440), (33, 3155), (0, 3223), (29, 3155), (12, 3155), (3, 3224), (10, 3155), (8, 3155), (0, 3338), (49, 3210), (46, 3155), (42, 992)]
960000: minimum gate count is 52, correctness: True
Best graph trace: [(28, 3223), (22, 3287), (7, 440), (11, 3224), (9, 3224), (3, 1777), (21, 1173), (0, 3223), (19, 440), (24, 3223), (3, 3224), (33, 440), (19, 440), (0, 3223), (37, 3155), (19, 440), (35, 3155), (3, 3224), (19, 440), (33, 3155), (0, 3223), (29, 3155), (12, 3155), (3, 3224), (10, 3155), (8, 3155), (0, 3338), (49, 3210), (46, 3155), (42, 992), (49, 3123), (47, 1119), (47,

KeyboardInterrupt: 

In [9]:
# Collecting data
import heapq
from concurrent.futures import ProcessPoolExecutor
import copy
import random

candidate_hq = []
heapq.heappush(candidate_hq, my_graph)
hash_set = set()
hash_set.add(my_graph.hash())
best_graph = my_graph
best_gate_cnt = my_graph.gate_count
budget = 1_000_000
collected_graphs = []


while candidate_hq != [] and budget >= 0:
    first_candidate = heapq.heappop(candidate_hq)
    all_nodes = first_candidate.all_nodes()
    
    def ax(i):
        node = all_nodes[i]
        return first_candidate.available_xfers(context=quartz_context, node=node)
    
    with ProcessPoolExecutor(max_workers=24) as executor:
        results = executor.map(ax, list(range(len(all_nodes))))
        appliable_xfers_nodes = []
        for r in results:
            appliable_xfers_nodes.append(r)
        
    for i in range(len(all_nodes)):
        node = all_nodes[i]
        appliable_xfers = appliable_xfers_nodes[i]
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            if new_graph.hash() not in hash_set:
                hash_set.add(new_graph.hash())
                heapq.heappush(candidate_hq, new_graph)
                if random.random() < 0.05:
                    collected_graphs.append(new_graph)
                    if len(collected_graphs) >= 2000:
                        break
                if new_graph < best_graph:
                    best_graph = new_graph
                    best_gate_cnt = new_graph.gate_count
                budget -= 1
                if budget % 10_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}, correctness: {check(best_graph)}')
    if len(collected_graphs) >= 2000:
        collected_graphs = collected_graphs[:2000]
        break


990000: minimum gate count is 58, correctness: True
980000: minimum gate count is 58, correctness: True
970000: minimum gate count is 58, correctness: True
960000: minimum gate count is 56, correctness: True


In [11]:
def save_to_qasm(i):
    collected_graphs[i].to_qasm(filename=f'barenco_graphs/barenco_tof_3_{i}.qasm')
    return True

with ProcessPoolExecutor(max_workers=24) as executor:
    results = executor.map(save_to_qasm, list(range(2000)), chunksize=64)

for r in results:
    pass

In [ ]:
check(best_graph)

In [ ]:
# 840000: minimum gate count is 42, correctness: True
# Best graph trace: [(28, 3223), (28, 3223), (8, 3430), (33, 440), (13, 2281), (37, 3155), (35, 3155), (34, 992), (21, 3287), (32, 440), (8, 440), (40, 3224), (49, 3210), (46, 3155), (28, 3338), (26, 440), (49, 3123), (31, 440), (47, 1119), (44, 3211), (39, 440), (8, 440), (46, 3212), (47, 3123), (42, 3224), (22, 3430), (45, 1119), (33, 440), (5, 3224), (8, 440), (33, 440), (24, 3287), (45, 3123), (43, 1119), (41, 440), (26, 3166), (24, 988), (1, 3224), (33, 440), (20, 3224), (32, 1241), (29, 375), (26, 2403), (0, 3223), (18, 3223), (25, 1289), (12, 440), (7, 3209), (6, 929), (1, 3224), (4, 440), (25, 1405), (28, 1602), (40, 1242), (26, 995), (24, 988), (0, 3223), (12, 3156), (23, 1405), (26, 1602), (27, 1242), (24, 995), (23, 1634), (11, 440), (10, 988), (1, 3224), (4, 440), (9, 3223), (17, 3224), (13, 3224), (6, 1447), (5, 1512), (4, 1967), (3, 988)]